# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Defaulting to user installation because normal site-packages is not writeable


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [4]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [5]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [6]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

D:\soft\miniConda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [8]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [10]:
print(destinations)

['生物: 适用于回答生物学科的相关问题', '计算机: 适用于回答计算机学科的相关问题', '汉语言文学: 适用于回答汉语言文学的相关问题']


In [11]:
print(destinations_str)

生物: 适用于回答生物学科的相关问题
计算机: 适用于回答计算机学科的相关问题
汉语言文学: 适用于回答汉语言文学的相关问题


In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
生物: 适用于回答生物学科的相关问题
计算机: 适用于回答计算机学科的相关问题
汉语言文学: 适用于回答汉语言文学的相关问题

<< INPUT >>
{input}

<< OUTPUT (must 

In [14]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': 'What is blackbody radiation?'}
> Finished chain.
{'input': 'What is blackbody radiation?', 'text': '\n\n黑体辐射是一种物理现象，指的是由于物体内部的热运动导致的电磁辐射。简单来说，物体在受热时会发出电磁波，这些波的频率和强度取决于物体的温度。黑体辐射的特点是它的辐射频率与物体的温度无关，而只与物体的结构和性质有关。这个概念在热力学和量子力学中都有重要的应用。'}


In [16]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要确定大于40的质数。40不是质数，我们从41开始检查。41是质数，但41+1=42不能被3整除。接下来是43，43+1=44也不能被3整除。然后是47，47+1=48可以被3整除。所以大于40的第一个质数是47，47+1=48可以被3整除。'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度和强大引力的天体，它的重力场非常强大，甚至连光也无法逃脱。它的存在是由于恒星在死亡时发生了坍缩，使得它们的质量集中在极小的空间内。黑洞的存在已经被多项观测和理论证据所证实，但仍然有很多未解之谜。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [1]:
biology_template = """你是一位很棒的生物学家。你擅长回答生物学科的相关问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_template = """你是一位很棒的计算机教授。你擅长回答计算机学科的相关问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

chinese_philology_template = """你是一位很棒的汉语言文学教授。你擅长回答汉语言文学的相关问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [2]:
prompt_infos = [
    {
        "name": "生物",
        "description": "适用于回答生物学科的相关问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机学科的相关问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学的相关问题",
        "prompt_template": chinese_philology_template,
    },
]

In [15]:
print(chain.invoke("植物细胞中都有叶绿体吗?"))



> Entering new MultiPromptChain chain...
生物: {'input': '植物细胞中都有叶绿体吗?'}
> Finished chain.
{'input': '植物细胞中都有叶绿体吗?', 'text': '\n\n是的，大多数植物细胞都有叶绿体，它们是进行光合作用的重要器官。但也有一些植物细胞缺乏叶绿体，例如在黑暗中生长的植物或者寄生植物。'}


In [16]:
print(chain.invoke("什么是基于区域的防火墙?"))



> Entering new MultiPromptChain chain...
计算机: {'input': '什么是基于区域的防火墙?'}
> Finished chain.
{'input': '什么是基于区域的防火墙?', 'text': '\n\n基于区域的防火墙是一种网络安全措施，它通过对网络进行分区，将网络分割为不同的区域，每个区域有自己的访问控制策略，从而实现对网络流量的控制和管理。这种防火墙可以根据不同的安全级别对不同的区域进行不同程度的保护，从而有效地防止未经授权的访问和恶意攻击。'}


In [17]:
print(chain.invoke("从自然属性角度划分出来的最小语音单位是什么?"))



> Entering new MultiPromptChain chain...
None: {'input': '从自然属性角度划分出来的最小语音单位是什么?'}
> Finished chain.
{'input': '从自然属性角度划分出来的最小语音单位是什么?', 'history': '', 'text': ' 从自然属性角度划分出来的最小语音单位是音素。音素是语言学中最小的语音单位，它是指在发音过程中不可再分的最小音素。每种语言中的音素数量不同，通常在20至60个之间。'}


In [18]:
print(chain.invoke("汉语拼音方案是以什么字母为根底制订的?"))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '汉语拼音方案是以什么字母为根底制订的?'}
> Finished chain.
{'input': '汉语拼音方案是以什么字母为根底制订的?', 'text': '\n\n汉语拼音方案是以拉丁字母为根底制订的。中国教育部于1958年公布了第一批汉语拼音方案，采用了拉丁字母来表示汉语的音节，每个音节由一个辅音和一个或多个元音组成。这种拼音方案的制订旨在方便汉语学习者学习和使用汉语，同时也为汉语的国际传播提供了便利。目前，汉语拼音方案已经成为中国官方推荐的拼音方法，并被广泛应用于汉语教学和文字输入。'}


In [19]:
print(chain.invoke("光合作用如何影响生物的呼吸和生长发育?"))



> Entering new MultiPromptChain chain...
生物: {'input': '光合作用如何影响生物的呼吸和生长发育?'}
> Finished chain.
{'input': '光合作用如何影响生物的呼吸和生长发育?', 'text': '\n\n光合作用是一种生物化学过程，通过光能将二氧化碳和水转化为能量丰富的有机物质（如葡萄糖），同时释放出氧气。这种过程对生物的呼吸和生长发育有着重要的影响。\n\n首先，光合作用产生的氧气是生物呼吸过程中必需的。在光合作用中，植物和其他光合生物通过光合作用产生的氧气来进行呼吸，从而将有机物转化为能量。呼吸过程中，氧气与有机物反应，产生能量和二氧化碳。因此，光合作用直接影响生物的呼吸过程，保证生物能够进行正常的代谢活动。\n\n其次，光合作用产生的有机'}
